# Example of BOMD MD (NVE, NVT)

In [1]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")

import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Decorating your function! <function KSA_XL_BOMD.one_step at 0x7f645632a790>


In [2]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   562.77   2.909726e-01 -1.364080e+00 -1.073107e+00 ||    210.05   1.086065e-01 1.426851e+00 1.535457e+00 || 
     2   537.09   2.776984e-01 -1.350657e+00 -1.072959e+00 ||    246.38   1.273909e-01 1.407842e+00 1.535232e+00 || 
     3   485.90   2.512280e-01 -1.323939e+00 -1.072711e+00 ||    302.32   1.563108e-01 1.378571e+00 1.534882e+00 || 
     4   416.33   2.152570e-01 -1.287662e+00 -1.072405e+00 ||    368.71   1.906365e-01 1.343809e+00 1.534446e+00 || 
     5   337.35   1.744240e-01 -1.246514e+00 -1.072090e+00 ||    434.22   2.245069e-01 1.309479e+00 1.533986e+00 || 
     6   258.22   1.335114e-01 -1.205345e+00 -1.071834e+00 ||    487.20   2.519035e-01 1.281676e+00 1.533579e+00 || 
     7   187.13   9.675234e-02 -1.168441e+00 -1.071688e+00 ||    518.00   2.678274e-01 1.265481e+00 1.533308e+00 || 
     8   130.03   6.722996e-02 -1.138860e+00 -1.071630e+00 ||    521.16   2.694583e-01 1.26377

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created. Coordinates, velocities, forces and charges are recorded in the following sequence: x y z vx vy vz fx fy fz q.

## *MD_BOMD_BASIC.Info.0.txt* and *MD_BOMD_BASIC.Info.1.txt* contain additional info for each step (orbital energies, dipole).

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [3]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_E_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   331.23   1.712617e-01 -6.518024e-02 1.060815e-01 ||    285.13   1.474213e-01 2.071960e+00 2.219381e+00 || 
     2   325.11   1.680973e-01 -6.201581e-02 1.060815e-01 ||    507.37   2.623320e-01 1.957049e+00 2.219381e+00 || 
     3   371.63   1.921463e-01 -8.606480e-02 1.060815e-01 ||    793.00   4.100139e-01 1.809367e+00 2.219381e+00 || 
     4   475.13   2.456623e-01 -1.395808e-01 1.060815e-01 ||   1118.98   5.785589e-01 1.640822e+00 2.219381e+00 || 
     5   637.26   3.294909e-01 -2.234095e-01 1.060815e-01 ||   1452.93   7.512242e-01 1.468157e+00 2.219381e+00 || 
     6   855.74   4.424532e-01 -3.363717e-01 1.060815e-01 ||   1752.69   9.062129e-01 1.313168e+00 2.219381e+00 || 
     7  1123.42   5.808509e-01 -4.747694e-01 1.060815e-01 ||   1967.62   1.017339e+00 1.202042e+00 2.219381e+00 || 
     8  1427.58   7.381148e-01 -6.320333e-01 1.060815e-01 ||   2043.95   1.056802e+00 1.162580e+00 2.

## Scale velocities to control temperature

In [4]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_T_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   2.068160e-01 -1.186363e-02 1.949524e-01 ||    400.00   2.068160e-01 2.186814e+00 2.393630e+00 || 
     2   400.00   2.068160e-01 3.204960e-02 2.388656e-01 ||    400.00   2.068160e-01 2.191146e+00 2.397962e+00 || 
     3   400.00   2.068160e-01 5.558028e-02 2.623963e-01 ||    400.00   2.068160e-01 2.154663e+00 2.361479e+00 || 
     4   400.00   2.068160e-01 5.275909e-02 2.595751e-01 ||    400.00   2.068160e-01 2.085426e+00 2.292242e+00 || 
     5   400.00   2.068160e-01 1.820372e-02 2.250197e-01 ||    400.00   2.068160e-01 1.994933e+00 2.201749e+00 || 
     6   400.00   2.068160e-01 -4.921782e-02 1.575982e-01 ||    400.00   2.068160e-01 1.893530e+00 2.100346e+00 || 
     7   400.00   2.068160e-01 -1.441194e-01 6.269661e-02 ||    400.00   2.068160e-01 1.788786e+00 1.995602e+00 || 
     8   400.00   2.068160e-01 -2.565661e-01 -4.975014e-02 ||    400.00   2.068160e-01 1.685829e+00 1.892

## Langevin Thermostat

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Langevin'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   353.46   1.827534e-01 -1.358699e+00 -1.175945e+00 ||    379.02   1.959708e-01 1.498579e+00 1.694550e+00 || 
     2   324.89   1.679815e-01 -1.345874e+00 -1.177893e+00 ||    257.96   1.333752e-01 1.548020e+00 1.681395e+00 || 
     3   276.14   1.427738e-01 -1.326710e+00 -1.183937e+00 ||    191.61   9.906993e-02 1.581930e+00 1.681000e+00 || 
     4   223.36   1.154839e-01 -1.305012e+00 -1.189528e+00 ||    179.74   9.293357e-02 1.594758e+00 1.687691e+00 || 
     5   187.08   9.672554e-02 -1.282685e+00 -1.185960e+00 ||    209.93   1.085402e-01 1.586147e+00 1.694687e+00 || 
     6   154.21   7.973507e-02 -1.262094e+00 -1.182359e+00 ||    264.09   1.365459e-01 1.558987e+00 1.695533e+00 || 
     7   136.96   7.081249e-02 -1.243525e+00 -1.172712e+00 ||    335.74   1.735921e-01 1.520398e+00 1.693990e+00 || 
     8   124.82   6.453435e-02 -1.227091e+00 -1.162557e+00 ||    429.95   2.223005e-01 1.47429